## Debugging q Code

First part of this chapter we make a brief overview of the native errors we may encounter when using incorrect syntax.

Then we focus on how to debug the code when encountering errors and move on to advise on how we can structure our codebase better.

### 5.1 Introduction to making it wrong: Errors

2 main pain points:

    - Avoiding simple syntax errors when initially writing our code.
    - interpreting the generally obscure and short q errors and how we can make these more verbose while our code increases in complexity